# 📊 Analiza danych nieruchomości
Celem projektu jest zebranie danych ze strony Gratka.pl oraz ich analiza: tytuł, opis ceny, metraż. Zebrane dane będa z Wielkopolski.

1.Import bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

 2. Web scraping

In [ ]:
dane = [] 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
}

base_url = 'https://gratka.pl/nieruchomosci/mieszkania/wielkopolskie'
start_url = f"{base_url}?sort=relevance"

response = requests.get(start_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

pagination_links = soup.select('a.pagination__item')
page_numbers = [int(link.text.strip()) for link in pagination_links if link.text.strip().isdigit()]
max_page = max(page_numbers) if page_numbers else 1

for page in range(1, max_page + 1):
    if page == 1:
        url = start_url
    else:
        url = f'{base_url}?page={page}&sort=relevance'

    print(f"Pobieram stronę {page}/{max_page}: {url}")
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        #Tytuły
        tytuly = [t.text for t in soup.find_all(class_="ehj8iw")]
        #Opisy
        opisy = [
            o.text.strip.split("Dodane")[0].strip().replace("Zobacz opis", "")
            for o in soup.find_all(class_="ORSCPR")
        ]
        #Ceny
        ceny = []
        for c in soup.find_all(class_="vQszq7"):
            tekst = c.text.replace(" ", "")
            match = re.match(r"(\d+)zł/m²(\d+)zł", tekst)
            if match:
                ceny.append(int(match.group(2)))
        #Metraże
        metraze = []
        for m in soup.find_all(class_="oaZarI"):
            match = re.search(r"(\d+)\s?m²", m.text)
            if match:
                metraze.append(int(match.group(1)))

        # Zgranie danych z mozliwioscia dodania pustych pol
        max_len = max(len(tytuly), len(opisy), len(ceny), len(metraze))
        for i in range(max_len):
            dane.append({
                'Tytuł': tytuly[i] if i < len(tytuly) else None,
                'Opis': opisy[i] if i < len(opisy) else None,
                'Cena': ceny[i] if i < len(ceny) else None,
                'Metraż': metraze[i] if i < len(metraze) else None
            })
    except Exception as e:
        print(f"Bład na stronie {page}: {e}")

    # Opoznienie miedzy zapytania 
    time.sleep(1.5)

df_gratka = pd.DataFrame(dane)
df_gratka.to_csv('gratka_mieszkania.csv', index=False)

 3. Pobieranie zbioru danych z Kaggle (polskie nieruchomosci)

In [ ]:
df_kaggle = pd.read_csv("data/polska_mieszkania.csv", sep=';', skiprows=1)

4. Przygotowanie danych 

In [ ]:
df_kaggle = df_kaggle[['address', 'city', 'price', 'sq']]
df_kaggle = df_kaggle.rename(columns={
    'address': 'Tytuł',
    'city': 'Opis',
    'price': 'Cena',
    'sq': 'Metraż'
})

# Usuwam duplikaty ofert
df_gratka.drop_duplicate(subset=['Tytuł', 'Cena', 'Metraż'], keep=first, inplace=True)
df_kaggle.drop_duplciate(subset=['Tytuł', 'Cena', 'Metraż'], keep=first, inplace=True)

# Usuwam oferty, które nie maja podanej ceny lub metrażu
df_gratka.dropna(subset=['Cena', 'Metraż'], how='any', inplace=True)
df_kaggle.dropna(subset=['Cena', 'Metraż'], how='any', inplace=True)

# Standaryzacja zmiennej - cena 
#print(df_gratka['Cena'].dtype)
#print(df_kaggle['Cena'].dtype)
df_gratka['Cena'] = df_gratka['Cena'].round(0).astype('Int64')
df_kaggle['Cena'] = df_kaggle['Cena'].round(0).astype('Int64')

# Standaryzacja zmiennej - metraż
#print(df_gratka['Metraż'].dtype)       - int64
#print(df_kaggle['Metraż'].dtype)       - float64
df_gratka['Metraż'] = df_gratka['Metraż'].round(2).astype('Float64')

# Oczyszanie zmiennych tytul i opis z zbednych znaków 
def oczyszczony_tekst(tekst):
    tekst = re.sub(r'<[^>]+>', '', tekst)
    tekst = re.sub(r'[^\w\sąćęłńóśżźĄĆĘŁŃÓŚŻŹ]', '', tekst)
    tekst = re.sub(r'\s+', '', tekst)
    return tekst.strip()

df_gratka['Tytuł'] = df_gratka['Tytuł'].astype(str).apply(oczyszczony_tekst)
df_gratka['Opis'] = df_gratka['Opis'].astype(str).apply(oczyszczony_tekst)

df_kaggle['Tytuł'] = df_kaggle['Tytuł'].astype(str).apply(oczyszczony_teskt)
df_kaggle['Opis'] = df_kaggle['Opis'].astype(str).apply(oczyszczony_tekst)

# Zidentikuje i usune podejrzane oferty 
min_price = 100000
max_price = 10000000
min_sq = 10
max_sq = 200

df_gratka = df_gratka[
    (df_gratka['Cena'] >= min_price) &
    (df_gratka['Cena'] <= max_price) &
    (df_gratka['Metraż'] >= min_sq) &
    (df_gratka['Metraż'] <= max_sq)
]

df_kaggle = df_kaggle[
    df_kaggle['Cena'] >= min_price &
    df_kaggle['Cena'] <= max_price &
    df_kaggle['Metraż'] >= min_sq &
    df_kaggle['Metraż'] <= max_sq
]


5. Zapisanie danych do CSV

In [ ]:
df_gratka.to_csv("gotowe_nieruchomosci.csv", index=False)
df_kaggle.to_csv("gotowe_kaggle.csv", index=False)